[Signature Analysis Pipeline - Readme on github](https://github.com/KnowEnG/Signature_Analysis_Pipeline/blob/master/README.md)

# KnowEnG 's Signiture Analysis Pipeline notebook


In [1]:
# %%writefile Signature_Analysis_notebook_tools.py
import os
import sys

from IPython.display import display
import ipywidgets as widgets

import knpackage.toolbox as kn

sys.path.insert(1, '../Signature_Analysis_Pipeline/src')
import gene_signature_toolbox as gsa_tbx

sys.path.insert(1, '../notebooks_KnowEnG/src')
from layout_notebooks import *

relative_path_root = '../Signature_Analysis_Pipeline'
run_files_path = '../Signature_Analysis_Pipeline/data/run_files'
# run_files_path = 'user_data/run_files'
results_dir = 'results'

spreadsheets_dir = os.path.join(os.path.join(relative_path_root, 'data'), 'spreadsheets')
networks_dir = os.path.join(os.path.join(relative_path_root, 'data'), 'networks')
run_files_dir = os.path.join(os.path.join(relative_path_root, 'data'), 'run_files')

local_dict = {'data/spreadsheets': spreadsheets_dir, 
              'data/networks': networks_dir, 
              'run_dir' : '.', 
              'data/run_files' : run_files_dir}


def visualize_selected_yaml_file(button):
    """ callback for yaml file view / clear button """
    if button.description == 'Clear':
        button.view_box.value = ''
        button.description = 'View'
    else:
        yaml_files_directory = button.file_selector.data_directory
        yaml_file_name = button.file_selector.value
        button.view_box.value = get_run_parameters_string(yaml_files_directory, yaml_file_name)
        button.description = 'Clear'
    
    
def get_run_parameters_string(run_files_path, yaml_file_name):
    run_parameters = kn.get_run_parameters(run_files_path, yaml_file_name)
    S = '<p>'
    for k, v in run_parameters.items():
        S += '%25s : %s'%(k, v) + '<br>'
    S += '</p>'
    return S

def get_yaml_file_button_set(data_directory, button_name='View'):
    """ get a view button with file select listbox and a file view box """
    select_file_button = widgets.Button(description=button_name,
                                           disabled=False,
                                           button_style='',
                                           tooltip='visualize selected file')

    select_file_button.view_box = get_view_box()
    select_file_button.file_selector = get_dropdown_files_listbox(data_directory, file_types=['.yml'])
    select_file_button.file_selector.data_directory = data_directory
    select_file_button.on_click(visualize_selected_yaml_file)

    return  select_file_button

def disp_run_parameters(run_parameters):
    """ formateed display of the run_parameters dict """
    for k, v in run_parameters.items():
        print('%25s : %s'%(k, v))

def set_local_run_parameters(run_parameters, local_dict):
    """ run_parameters = set_local_run_parameters(run_parameters, local_dict) """
    for key_name, key_value in local_dict.items():
        for k, v in run_parameters.items():
            if 'full_path' in k or 'directory' in k:
                if key_name in v:
                    de_nada, f_name = os.path.split(v)
                    run_parameters[k] = os.path.join(key_value, f_name)
                
    return run_parameters

def run_yaml_file(run_files_path, yaml_file_name):
    if os.path.isfile(os.path.join(run_files_path, yaml_file_name)):
        run_parameters = kn.get_run_parameters(run_files_path, yaml_file_name)

        run_parameters['results_directory'] = results_dir
        run_parameters['tmp_directory'] = 'tmp'
        run_parameters['run_directory'] = '.'
        run_parameters = set_local_run_parameters(run_parameters, local_dict)
        
        if run_parameters['method'] == 'similarity':
            gsa_tbx.run_similarity(run_parameters)
        elif run_parameters['method'] == 'cc_similarity':
            gsa_tbx.run_cc_similarity(run_parameters)
        elif run_parameters['method'] == 'net_similarity':
            gsa_tbx.run_net_similarity(run_parameters)
        elif run_parameters['method'] == 'cc_net_similarity':
            gsa_tbx.run_cc_net_similarity(run_parameters)
        else:
            print('run_parameters method not found')
            
def execute_yaml_file(button):
    """  """
    yaml_files_directory = button.file_selector.data_directory
    yaml_file_name = button.file_selector.value

    run_yaml_file(yaml_files_directory, yaml_file_name)
    

In [2]:
select_file_button = get_yaml_file_button_set(run_files_path)
show_select_view_button(select_file_button)

Box(children=(Dropdown(layout=Layout(width='50%'), options=('BENCHMARK_1_GS_cos.yml', 'BENCHMARK_1_GS_spearman.yml', 'BENCHMARK_2_GS_net_cos.yml', 'BENCHMARK_2_GS_net_spearman.yml', 'BENCHMARK_3_GS_cc_cos.yml', 'BENCHMARK_3_GS_cc_spearman.yml', 'BENCHMARK_4_GS_cc_net_cos.yml', 'BENCHMARK_4_GS_cc_net_spearman.yml', 'TEST_1_GS_cos.yml', 'TEST_1_GS_spearman.yml', 'TEST_2_GS_net_cos.yml', 'TEST_2_GS_net_spearman.yml', 'TEST_3_GS_cc_cos.yml', 'TEST_3_GS_cc_spearman.yml', 'TEST_4_GS_cc_net_cos.yml', 'TEST_4_GS_cc_net_spearman.yml', 'zTEMPLATE_cc_net_cos.yml', 'zTEMPLATE_cc_net_spearman.yml'), value='BENCHMARK_1_GS_cos.yml'), Button(description='View', style=ButtonStyle(), tooltip='visualize selected file')), layout=Layout(align_items='stretch', border='none', display='inline-flex', flex_flow='row', justify_content='space-between', width='100%'))

HTML(value='')

In [4]:
exbutton = get_single_file_execute_button(run_files_path, 
                                          results_dir, 
                                          select_file_button.file_selector, 
                                          button_name='RUN')
exbutton.on_click(execute_yaml_file)
show_execute_button(exbutton)

HBox(children=(Button(description='RUN', style=ButtonStyle(), tooltip='execute selected file'),), layout=Layout(display='flex', flex_flow='row', justify_content='flex-end', width='100%'))

HTML(value='')

In [ ]:
# Choose a yaml file
# import sys
# sys.path.insert(1, '../notebooks_KnowEnG/src')
# from Signature_Analysis_notebook_tools import *

yaml_file_name = 'BENCHMARK_2_GS_net_cos.yml'
S = get_run_parameters_string(run_files_path, yaml_file_name)
view_text_box.value = S


# run_yaml_file(yaml_file_name)